In [91]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import IncrementalPCA
from tabulate import tabulate
import tensorflow as tf
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

## Initial Loading

In [93]:

ddi_fp = "drugbank\drugbank.tab"
ddi = pd.read_csv(ddi_fp, sep='\t')

kaggle_fp = "SMILES-Kaggle\chembl_22_clean_1576904_sorted_std_final.smi"
smiles = pd.read_csv(kaggle_fp, sep='\t')

drug_names_fp = "drugs.txt"
drug_names = pd.read_csv(drug_names_fp, sep='\t')

ddi["Y"] = ddi["Y"].astype("category")
ddi["Map"] = ddi["Map"].astype("category")

#counting interaction types for potential later weighting
interaction_counts = pd.DataFrame(ddi['Y'].value_counts().rename_axis('value').reset_index(name='count')).sort_values(by='count', ascending=False)
interaction_counts['row_num'] = interaction_counts.index + 1
interaction_counts['log_count'] = np.log(interaction_counts['count'])

#listing longer explanations of interaction types for later use
interaction_types = ddi[['Y','Map']].drop_duplicates(subset=['Y'])

#remove longer name of interaction type from main dataset
ddi = ddi.drop("Map",axis=1)

ddi.head(10)

,ID1,ID2,Y,X1,X2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
5,DB11630,DB00460,1,OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
6,DB00553,DB00460,1,COC1=C2OC(=O)C=CC2=CC2=C1OC=C2,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
7,DB06261,DB00460,1,[H]N([H])CC(=O)CCC(=O)OCCCCCC,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
8,DB01878,DB00460,1,O=C(C1=CC=CC=C1)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
9,DB00140,DB00460,1,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...


In [94]:
ddi.shape

(191808, 5)

In [95]:
#quick function to turn a list of size 1 lists of strings into a list of strings, for later use
def delist(list_of_lists):
    list_of_strings = []
    for inner_list in list_of_lists:
        string = inner_list[0]
        list_of_strings.append(string)
    return list_of_strings

## Preprocessing

In [97]:

# counting drugs by number of mentions in database
old = pd.DataFrame()
old["total"] = ddi['ID1'].value_counts()
old = old.reset_index()
old.columns = ['ID', 'count'] 
new = pd.DataFrame()
new["total"] = ddi['ID2'].value_counts()
new = new.reset_index()
new.columns = ['ID', 'count'] 
drug_counts = pd.merge(old,new,how='outer',on='ID').fillna(0)
drug_counts['total'] = drug_counts['count_x'] + drug_counts['count_y']

drug_counts = drug_counts.sort_values(by='total')
drug_counts_one = pd.DataFrame(drug_counts[drug_counts['total']==1]['ID'])

#removing drugs only in database once
ddi_proc = ddi[ ~ddi['ID1'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ ~ddi_proc['ID2'].isin(drug_counts_one['ID'])]

#removing one particular drug with a problematic SMILES code
ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


In [98]:
#create main datasets

data = dc.data.NumpyDataset(X=ddi_proc[['X1','X2']], y=ddi[['Y']])
df = data.to_dataframe()
df = df.sample(frac=1).reset_index(drop=True)

X_one = delist(df[["X1"]].values.tolist())
X_two = delist(df[["X2"]].values.tolist())


search_string = "nan"

count = X_one.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_one.")
count = X_two.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_two.")

## Featurization and Dimensionality Reduction

In [100]:
#featurize using circular fingerprint

cf_featurizer = dc.feat.CircularFingerprint()

def circular_fingerprint(smiles):
    try:
        mol = cf_featurizer(smiles)
        return mol
    except Exception as e:
        print(f"Error fingerprinting {smiles}: {e}")
        return None  # Skipping invalid SMILES


#other way to featurize a molecule
#cm_featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

#example process with just 10000 rows
df_example = df.head(10000)
df_example = df_example.reset_index()
df_example.rename(columns={'index':'col'},  inplace=True)

df_example_cf = pd.DataFrame()
df_example_cf['x1_cf'] = df_example['X1'].apply(circular_fingerprint)
df_example_cf['x2_cf'] = df_example['X2'].apply(circular_fingerprint)
df_example_cf['col'] = df_example_cf.index
df_example_cf.dropna(subset=['x1_cf', 'x2_cf'], inplace=True)


df_example_x1_cf = pd.DataFrame(delist(df_example_cf['x1_cf']))
df_example_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
df_example_x1_cf['col'] = df_example_cf['col']

df_example_x2_cf = pd.DataFrame(delist(df_example_cf['x2_cf']))
df_example_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
df_example_x2_cf['col'] = df_example_cf['col']


df_example = df_example.merge(df_example_x1_cf, on="col")
df_example = df_example.merge(df_example_x2_cf, on="col")

df_example

[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerator
[19:45:34] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerator
[19:45:49] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerator
[19:45:50] DEPRECATION WARNING: please use MorganGenerat

...

Error fingerprinting nan: 'float' object is not iterable


[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerator
[19:46:11] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[19:46:16] DEPRECATION WARNING: please use MorganGenerator
[19:46:16] DEPRECATION WARNING: please use MorganGenerator


,col,X1,X2,y,w,ids,x1_cf_1,x1_cf_2,x1_cf_3,x1_cf_4,...,x2_cf_2039,x2_cf_2040,x2_cf_2041,x2_cf_2042,x2_cf_2043,x2_cf_2044,x2_cf_2045,x2_cf_2046,x2_cf_2047,x2_cf_2048
0,0,CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1C...,COC(=O)C1=C(C)NC(C)=C(C1C1=CC=CC=C1[N+]([O-])=...,49,1.0,110115,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,O=[As]O[As]=O,OC(=O)P(O)(O)=O,20,1.0,24619,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,O.O.O.O.C(CNCC1=CC=CC=C1)NCC1=CC=CC=C1.[H][C@]...,CCCN(CCC)S(=O)(=O)C1=CC=C(C=C1)C(O)=O,73,1.0,162054,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,CC(C(O)=O)C1=CC=C(C=C1)C(=O)C1=CC=CS1,NS(=O)(=O)C1=C(Cl)C=C2NC=NS(=O)(=O)C2=C1,70,1.0,149232,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,CC(C)NCC(O)COC1=CC=C(CC(N)=O)C=C1,CC(=O)OCC[N+](C)(C)C,49,1.0,68837,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,9991,CN(C)CCC(C1=CC=C(Br)C=C1)C1=CC=CC=N1,CCN(CC)CCCC(C)NC1=CC=NC2=CC(Cl)=CC=C12,47,1.0,39214,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9988,9992,CC1=CC=C(SC2=CC=CC=C2N2CCNCC2)C(C)=C1,CS(=O)(=O)CCNCC1=CC=C(O1)C1=CC2=C(C=C1)N=CN=C2...,47,1.0,63868,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9989,9993,CCCC(C)C1(CC)C(=O)NC(=O)NC1=O,CC(C)OC(=O)C(C)(C)OC1=CC=C(C=C1)C(=O)C1=CC=C(C...,4,1.0,4781,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9990,9994,CC1=C(SC=C1)C(=CCCN1CCC[C@H](C1)C(O)=O)C1=C(C)...,CCN(CC)C(=O)C1(CC1CN)C1=CC=CC=C1,49,1.0,107919,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
#Basic dimensionality reduction down to 100 components from 4096

df_X = df_example.iloc[:,6:]
df_Y = to_categorical(df_example["y"])

n_components = 100
batch_size = 200

ipca = IncrementalPCA(n_components=n_components)

for i in range(0, df_X.shape[0], batch_size):
    X_batch = df_X[i:i + batch_size]
    ipca.partial_fit(X_batch)

X_transformed = ipca.transform(df_X)

#prove that X's shape has changed
#print("Original shape:", df_X.shape)
#print("Transformed shape:", X_transformed.shape)

df_X_proc = pd.DataFrame(X_transformed)


## Initial basic model implementation

In [103]:
#basic/rough neural network implementation

def calc_layers(X_size, Y_size):
    layers = [X_size+1]
    layer = 2
    while layer <= X_size:
        layer = int(layer * 2)
    layers.append(layer)
    if X_size > Y_size:
        while layer / 2 > Y_size and layer > 2:
            layer = layer / 2
            layers.append(int(layer))
    layers.append(Y_size)
    return layers
        
def DNN(X, Y, Epochs, batchsize, layernum):

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=27)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #define layers and nodes in each layer
    input = len(X.columns)
    output = Y.shape[1]
    if layernum=='many':
        layers = calc_layers(input,output)
    elif type(layernum)==int:
        layers = [input]
        for i in range(layernum, 1, -1):
            layer = int(round((i * (input + output) / (layernum+1)), 0))
            if layer > output:
                layers.append(layer)
        layers.append(output)
    else:
        print(f"incorrect layernum {layernum}")
        return None

    model = keras.models.Sequential()

    model.add(Dense(layers[0], activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    for layer_size in layers[1:-1]:
        model.add(Dense(layer_size, activation='relu'))
        
    model.add(Dense(layers[-1], activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=Epochs, batch_size=batchsize, validation_split=0.1)
    
    full_loss, full_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {full_accuracy}")
    print(f"Test Loss: {full_loss}")
    return model

In [104]:
dr_model = DNN(df_X_proc, df_Y, 10, 500, 1)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.0081 - loss: 0.6627 - val_accuracy: 0.0037 - val_loss: 0.4646
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0095 - loss: 0.4338 - val_accuracy: 0.0113 - val_loss: 0.2972
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0169 - loss: 0.2810 - val_accuracy: 0.0312 - val_loss: 0.1899
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0469 - loss: 0.1828 - val_accuracy: 0.1363 - val_loss: 0.1267
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1206 - loss: 0.1258 - val_accuracy: 0.2663 - val_loss: 0.0918
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2150 - loss: 0.0938 - val_accuracy: 0.3388 - val_loss: 0.0727
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2713 - loss: 0.0758 - val_accuracy: 0.3550 - val_loss: 0.0617
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2877 - loss: 0.0652 - val_accuracy: 0.3512 - v

In [105]:
#comparison to without Dimensionality reduction

full_model = DNN(df_X, df_Y, 10, 500, 1)

#calc_layers(len(df_X.iloc[:,1:].columns), df_Y.shape[1])

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 382ms/step - accuracy: 0.2368 - loss: 0.2658 - val_accuracy: 0.4875 - val_loss: 0.0473
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 369ms/step - accuracy: 0.6658 - loss: 0.0270 - val_accuracy: 0.5000 - val_loss: 0.0472
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 369ms/step - accuracy: 0.8240 - loss: 0.0138 - val_accuracy: 0.5100 - val_loss: 0.0466
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 369ms/step - accuracy: 0.9286 - loss: 0.0072 - val_accuracy: 0.4950 - val_loss: 0.0487
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 374ms/step - accuracy: 0.9687 - loss: 0.0046 - val_accuracy: 0.5075 - val_loss: 0.0494
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 371ms/step - accuracy: 0.9885 - loss: 0.0030 - val_accuracy: 0.5288 - val_loss: 0.0498
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 372ms/step - accuracy: 0.9941 - loss: 0.0022 - val_accuracy: 0.5088 - val_loss: 0.0518
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 395ms/step - accuracy: 0.9955 - loss: 0.0017 - val_accuracy: 0.

This demonstrates that dimensionality reduction is counterproductive to a neural network in this case.

## Adding in other variables

In [108]:
#code from Beth Farr
def compute_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None

        # Computing Molecular Descriptors
        mol_wt = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        h_donors = Descriptors.NumHDonors(mol)
        h_acceptors = Descriptors.NumHAcceptors(mol)
        tpsa = Descriptors.TPSA(mol)
        return [mol_wt, logp, h_donors, h_acceptors, tpsa]

    except Exception as e:
        print(f"Error computing features for {smiles}: {e}")
        return None  # Skipping invalid SMILES

# Using function to X1 and X2 to extract features
df_feat = df_example
df_feat['features_X1'] = df_example['X1'].apply(compute_features)
df_feat['features_X2'] = df_example['X2'].apply(compute_features)
df_feat = df_feat.dropna(subset=['features_X1', 'features_X2'])
features_X1_df = pd.DataFrame(df_feat['features_X1'].tolist(), columns=['MolWt_X1', 'LogP_X1', 'NumHDonors_X1', 'NumHAcceptors_X1', 'TPSA_X1'])
features_X1_df['col'] = df_feat['col']
features_X2_df = pd.DataFrame(df_feat['features_X2'].tolist(), columns=['MolWt_X2', 'LogP_X2', 'NumHDonors_X2', 'NumHAcceptors_X2', 'TPSA_X2'])
features_X2_df['col'] = df_feat['col']

df_feat = pd.merge(df_feat, pd.merge(features_X1_df, features_X2_df,on='col'), on='col')
df_feat = df_feat.drop(['X1', 'X2', 'w', 'features_X1', 'features_X2'], axis=1)

df_feat_X = df_feat.iloc[:,3:]
df_feat_Y = to_categorical(df_feat["y"])

feat_model = DNN(df_feat_X, df_feat_Y, 10, 500, 2)


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 549ms/step - accuracy: 0.1888 - loss: 0.2593 - val_accuracy: 0.3988 - val_loss: 0.0916
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 532ms/step - accuracy: 0.5453 - loss: 0.0594 - val_accuracy: 0.4963 - val_loss: 0.0545
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 534ms/step - accuracy: 0.7213 - loss: 0.0236 - val_accuracy: 0.4925 - val_loss: 0.0472
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 529ms/step - accuracy: 0.8410 - loss: 0.0127 - val_accuracy: 0.5200 - val_loss: 0.0482
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 531ms/step - accuracy: 0.9319 - loss: 0.0074 - val_accuracy: 0.5075 - val_loss: 0.0517
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 533ms/step - accuracy: 0.9669 - loss: 0.0048 - val_accuracy: 0.5200 - val_loss: 0.0552
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 534ms/step - accuracy: 0.9828 - loss: 0.0033 - val_accuracy: 0.5050 - val_loss: 0.0598
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 533ms/step - accuracy: 0.9851 - loss: 0.0027 - val_accuracy: 0